## Model Training

In [1]:
# Basic Import
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
# Modelling
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Ridge,Lasso
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.model_selection import RandomizedSearchCV
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
import warnings

## Import the CSV data as pandas dataframe 

In [2]:
df=pd.read_csv('well_data_standard.csv')


In [4]:
df.head()

,well_id,permeability_md,porosity_fraction,net_to_gross,thickness_ft,well_depth_ft,tubing_diameter_in,choke_size_64th,well_type,completion_type,artificial_lift,reservoir_pressure_psi,reservoir_temp_f,bottomhole_pressure_psi,wellhead_pressure_psi,oil_gravity_api,gas_oil_ratio_scf_bbl,water_cut_fraction,fvf_oil,oil_viscosity_cp,oil_rate_bbl_day,gas_rate_scf_day,water_rate_bbl_day,productivity_index,oil_price_usd_bbl,gas_price_usd_mcf,daily_opex_usd,drilling_cost_usd,completion_cost_usd,total_capex_usd,daily_revenue_usd,performance_index,oil_cut,profit_per_barrel,production_efficiency,economic_efficiency,ranking_score,drill_date,well_age_days,production_months,days_since_workover,pressure_drawdown,total_liquid_rate,productivity_factor,depth_category
0,1,12.142530,0.196309,0.731930,24.336113,7816.914302,2.875,40.952149,Horizontal,Slotted Liner,Gas Lift,4015.857729,203.266555,3297.690469,2638.152375,48.488745,89.184836,0.269383,1.102676,49.339189,7.678191,684.778161,2.830993,0.003975,76.230187,3.488500,375.195612,8.870844e+05,631088.272669,1.518173e+06,212.503134,113.674200,0.730617,27.320382,0.731617,0.566380,25.568573,2015-09-27,3384,22.669507,618.370405,718.167260,10.509183,2.383686,Medium
1,2,6.434896,0.245471,0.817931,133.211654,5886.339608,2.875,19.802734,Horizontal,Perforated,Rod Pump,3047.434493,171.874422,2445.322649,1956.258119,26.081809,106.131769,0.204494,1.103184,100.000000,5.000000,530.658845,1.285304,0.006356,79.560396,3.480281,322.648700,8.528049e+05,651547.966569,1.504353e+06,77.000121,97.776988,0.795506,15.098063,0.796506,0.238650,25.975528,2021-03-13,1390,40.224882,30.000000,602.111844,6.285304,1.579579,Medium
2,3,14.121360,0.080072,0.730540,73.854045,7387.046131,2.875,21.256347,Vertical,Perforated,Rod Pump,4084.408986,171.676602,3490.959928,2792.767942,44.461707,85.854177,0.271557,1.102576,72.438698,5.000000,429.270885,1.863953,0.009539,80.906653,3.930136,338.234414,1.293443e+06,486971.375072,1.780414e+06,67.985943,86.866010,0.728443,13.330577,0.729443,0.201002,23.867400,2023-12-06,392,31.995545,156.461027,593.449058,6.863953,1.130720,Medium
3,4,33.886946,0.178148,0.798838,55.712522,8718.228122,3.500,43.169706,Deviated,Perforated,Natural Flow,4773.433601,183.920459,4296.090241,3436.872193,25.738006,419.083092,0.155809,1.112572,100.000000,8.856987,3711.813362,1.634703,0.013556,70.886345,3.645415,310.959451,1.061199e+06,577026.652848,1.638226e+06,330.411064,169.899029,0.844191,36.888641,0.845191,1.062554,30.497943,2018-01-03,2555,3.784677,211.937914,477.343360,10.491690,6.036907,Deep
4,5,5.846520,0.117468,0.818555,33.298320,9019.363528,4.500,54.476617,Deviated,Perforated,Rod Pump,4743.758462,219.200751,4269.382616,3415.506093,36.194442,61.770762,0.234775,1.101853,80.128525,5.000000,308.853810,1.534027,0.001805,51.655844,3.298163,354.227663,9.286172e+05,419564.984217,1.348182e+06,-94.929792,55.638014,0.765225,-18.613685,0.766225,-0.267991,23.871175,2019-09-06,1944,46.007405,397.470614,474.375846,6.534027,0.686778,Deep


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 45 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   well_id                  500 non-null    int64  
 1   permeability_md          500 non-null    float64
 2   porosity_fraction        500 non-null    float64
 3   net_to_gross             500 non-null    float64
 4   thickness_ft             500 non-null    float64
 5   well_depth_ft            500 non-null    float64
 6   tubing_diameter_in       500 non-null    float64
 7   choke_size_64th          500 non-null    float64
 8   well_type                500 non-null    object 
 9   completion_type          500 non-null    object 
 10  artificial_lift          500 non-null    object 
 11  reservoir_pressure_psi   500 non-null    float64
 12  reservoir_temp_f         500 non-null    float64
 13  bottomhole_pressure_psi  500 non-null    float64
 14  wellhead_pressure_psi    5

In [20]:
df.shape

(500, 45)

In [70]:
X=df.drop(columns=['productivity_index'],axis=1)
y=df['productivity_index']

In [71]:
X.shape

(500, 44)

In [72]:
from sklearn.model_selection import train_test_split
# Select features for modeling
feature_columns = [
            'permeability_md', 'porosity_fraction', 'well_depth_ft', 'tubing_diameter_in',
            'choke_size_64th', 'reservoir_pressure_psi', 'oil_gravity_api', 'gas_oil_ratio_scf_bbl',
            'bottomhole_pressure_psi', 'water_cut_fraction', 'pressure_drawdown', 'productivity_factor'
        ]
        
X= df[feature_columns].copy()
y = df['performance_index'].copy()
        
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)


In [73]:
## Create eavaluavate function to give all metrics after model training

In [74]:
def evaluate_model(true,predicted):
    mae=mean_absolute_error(true,predicted)
    mse=mean_squared_error(true,predicted)
    rmse=np.sqrt(mean_squared_error(true,predicted))
    r2=r2_score(true, predicted)
    return mae,rmse,r2

In [75]:
models={
    'Linear Regression': LinearRegression(),
    'Lasso' : Lasso(),
    'Ridge': Ridge(),
    'K-Neighbors Regressor': KNeighborsRegressor(),
    'Decision Tree': DecisionTreeRegressor(),
    'Random Forest Regressor': RandomForestRegressor(),
    'XGBRegressor': XGBRegressor(),
    'CatBoostRegressor': CatBoostRegressor(),
    'AdaBoostRegressor': AdaBoostRegressor()

}

In [76]:
model_list=[]
r2_list=[]


In [77]:
for i in range(len(list(models))):
    model=list(models.values())[i]
    model.fit(X_train, y_train) # Train model

    # Make predictions
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    # Evaluate Train and Test dataset
    model_train_mae , model_train_rmse, model_train_r2 = evaluate_model(y_train, y_train_pred)

    model_test_mae , model_test_rmse, model_test_r2 = evaluate_model(y_test, y_test_pred)

    
    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])
    
    print('Model performance for Training set')
    print("- Root Mean Squared Error: {:.4f}".format(model_train_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_train_mae))
    print("- R2 Score: {:.4f}".format(model_train_r2))

    print('----------------------------------')
    
    print('Model performance for Test set')
    print("- Root Mean Squared Error: {:.4f}".format(model_test_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_test_mae))
    print("- R2 Score: {:.4f}".format(model_test_r2))
    r2_list.append(model_test_r2)
    
    print('='*35)
    print('\n')


Linear Regression
Model performance for Training set
- Root Mean Squared Error: 78.5835
- Mean Absolute Error: 38.9757
- R2 Score: 0.3577
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 110.8556
- Mean Absolute Error: 50.6632
- R2 Score: 0.2998


Lasso
Model performance for Training set
- Root Mean Squared Error: 79.4245
- Mean Absolute Error: 40.0514
- R2 Score: 0.3439
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 116.2032
- Mean Absolute Error: 52.2407
- R2 Score: 0.2306


Ridge
Model performance for Training set
- Root Mean Squared Error: 78.7529
- Mean Absolute Error: 39.0785
- R2 Score: 0.3549
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 113.0746
- Mean Absolute Error: 51.1776
- R2 Score: 0.2715


K-Neighbors Regressor
Model performance for Training set
- Root Mean Squared Error: 82.1052
- Mean Absolute Error: 39.9997
- R2 Score: 0.2988
------

In [25]:
y_train

249    0.001636
433    0.002120
19     0.000336
322    0.001481
332    0.003800
         ...   
106    0.016063
270    0.031822
348    0.000536
435    0.001708
102    0.001609
Name: productivity_index, Length: 400, dtype: float64

In [78]:
pd.DataFrame(list(zip(model_list, r2_list)), columns=['Model Name', 'R2_Score']).sort_values(by=["R2_Score"],ascending=False)


,Model Name,R2_Score
0,Linear Regression,0.299789
2,Ridge,0.271476
6,XGBRegressor,0.248022
1,Lasso,0.230604
7,CatBoostRegressor,0.221940
5,Random Forest Regressor,0.145563
8,AdaBoostRegressor,0.112145
3,K-Neighbors Regressor,-0.037176
4,Decision Tree,-1.219886


In [37]:
## Model training after data transformation

In [64]:
X1=df.drop(columns=['well_id','performance_index'])
y1=df.drop(columns=['performance_index'])

In [65]:
num_features=X1.select_dtypes(exclude='object').columns
cat_features=X1.select_dtypes(include='object').columns

from sklearn.preprocessing import OneHotEncoder,StandardScaler
from sklearn.compose import ColumnTransformer


num_transformer=StandardScaler()
oh_transformer=OneHotEncoder()
  

preprocessor = ColumnTransformer(
    [
        ("Onehotencoder",oh_transformer,cat_features),
        ('StandardScaler',num_transformer,num_features)
    ]
)

In [66]:
X1_t=preprocessor.fit_transform(X1)

In [67]:
# separate dataset into train and test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X1_t,y,test_size=0.2,random_state=42)
X_train.shape, X_test.shape

((400, 509), (100, 509))

In [68]:
for i in range(len(list(models))):
    model=list(models.values())[i]
    model.fit(X_train, y_train) # Train model

    # Make predictions
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    # Evaluate Train and Test dataset
    model_train_mae , model_train_rmse, model_train_r2 = evaluate_model(y_train, y_train_pred)

    model_test_mae , model_test_rmse, model_test_r2 = evaluate_model(y_test, y_test_pred)

    
    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])
    
    print('Model performance for Training set')
    print("- Root Mean Squared Error: {:.4f}".format(model_train_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_train_mae))
    print("- R2 Score: {:.4f}".format(model_train_r2))

    print('----------------------------------')
    
    print('Model performance for Test set')
    print("- Root Mean Squared Error: {:.4f}".format(model_test_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_test_mae))
    print("- R2 Score: {:.4f}".format(model_test_r2))
    r2_list.append(model_test_r2)
    
    print('='*35)
    print('\n')

Linear Regression
Model performance for Training set
- Root Mean Squared Error: 0.0038
- Mean Absolute Error: 0.0018
- R2 Score: 1.0000
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 12.2151
- Mean Absolute Error: 7.8150
- R2 Score: 0.9915


Lasso
Model performance for Training set
- Root Mean Squared Error: 6.2562
- Mean Absolute Error: 3.3559
- R2 Score: 0.9959
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 13.1639
- Mean Absolute Error: 4.7539
- R2 Score: 0.9901


Ridge
Model performance for Training set
- Root Mean Squared Error: 2.4710
- Mean Absolute Error: 1.4410
- R2 Score: 0.9994
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 7.8690
- Mean Absolute Error: 3.6466
- R2 Score: 0.9965


K-Neighbors Regressor
Model performance for Training set
- Root Mean Squared Error: 31.8534
- Mean Absolute Error: 14.2569
- R2 Score: 0.8945
-------------------

In [69]:
pd.DataFrame(list(zip(model_list, r2_list)), columns=['Model Name', 'R2_Score']).sort_values(by=["R2_Score"],ascending=False)

,Model Name,R2_Score
11,Ridge,0.996472
9,Linear Regression,0.991498
10,Lasso,0.990126
17,AdaBoostRegressor,0.929082
14,Random Forest Regressor,0.920200
13,Decision Tree,0.865116
15,XGBRegressor,0.864239
16,CatBoostRegressor,0.807954
12,K-Neighbors Regressor,0.650078
0,Linear Regression,0.299789
